In [149]:
import pandas as pd
import numpy as np

#from rapidfuzz import fuzz, process

## Import Data

Shopee data: April 2021

Lazada data:
- 10 kg > May 2020 - May 2021
- 5-10 kg > May 2020 - May 2021
- 0-5 kg > April 2021

In [150]:
spe = pd.read_csv("data\spe_items_2021-04-01_2021-04-30_plus_quant.csv")

In [151]:
spe.item_description = spe.item_description.str.lower()
spe = spe[spe.item_description.notnull()]
spe = spe.reset_index(drop=True)

In [152]:
laz_10kg = pd.read_csv("data\laz_items_2020-05-13_2021-05-13_10kg.csv")
laz_10kg.item_description = laz_10kg.item_description.str.lower()

In [153]:
laz_5kg = pd.read_csv("data\laz_items_2020-05-13_2021-05-13_5-10kg.csv")
laz_5kg.item_description = laz_5kg.item_description.str.lower()
#laz_5kg = laz_5kg.sample(n=4000, random_state=1)

In [154]:
laz_0kg_april = pd.read_csv("data\laz_items_2021-04-01_2021-04-30_0-5kg.csv")
laz_0kg_april.item_description = laz_0kg_april.item_description.str.lower()
#laz_0kg_april = laz_0kg_april.sample(n=10000, random_state=1)

Merge lazada data and take only 10000 samples (too big library will take a very long time to run)

But in the future we should really find ways to expand the library, because bigger library = much better accuracy

In [155]:
laz = pd.concat([laz_10kg, laz_5kg, laz_0kg_april])
laz = laz[laz.item_description.notnull()]
laz = laz.sample(n=10000, random_state=1)
laz = laz.reset_index(drop=True)

In [156]:
laz_items = laz.item_description.to_list()
laz_price = laz.item_price.to_list()
laz_weight = laz.final_weight.to_list()

In [350]:
spe.head()

,legacy_id,item_description,weight,price,quantity
0,303589,áo tập gym yoga nữ siêu xinh ami,0.15,190000,24
1,436058,"máy hút mụnmáy hút mụn đầu đenmàn hình led, má...",0.20,125000,73
2,437888,set 10 son nhung lì wetn,0.35,218000,1
3,358286,"quần kaki wellkids bé trai, quần short phối mầ...",0.12,184000,5
4,330220,gương mini cầm tay họa tiết hoạt hình dễ thươn...,0.21,113000,16


In [354]:
laz.head()

,item_description,item_price,final_weight
0,"khuếch tán tinh dầu purevess 9ml giúp khử mùi,...",52000.0,0.100000
1,"cốc gấu đựng bàn chải kem đánh răng, cốc gấu d...",14610.0,0.200000
2,"túi đeo chéo vải canvas,túi đeo chéo nữ,túi vả...",49000.0,0.300000
3,(có tem qr_icheck) thảo mộc khử mùi ngân bình ...,29000.0,0.200000
4,5 cái công tắc rocker 3 chân spdt bật/tắt/bật...,24000.0,0.166667


## Levenshtein Model

The below Levenshtein algo will find 5 best matched laz items for each spe item, plus matching score and the position of the laz item in the laz library

In [161]:
def fuzzy(laz_items, spe_items):
    matched_array = []
    ratio_array = []
    index_array = []
    
    for spe_item in spe_items:
        if spe_item in laz_items:
            matched_array.append([spe_item])
            ratio_array.append([100])
            index_array.append([999999])
        else:
            matches = process.extract(spe_item, laz_items, scorer=fuzz.token_sort_ratio, limit=5)
            
            matched = [x[0] for x in matches]
            ratio = [x[1] for x in matches]
            index = [x[2] for x in matches]
            
            matched_array.append(matched)
            ratio_array.append(ratio)
            index_array.append(index)
            
    return matched_array, ratio_array, index_array

Running the algo

In [158]:
#spe_items = spe.item_description.to_list()

In [162]:
#results = fuzzy(laz_items, spe_items)

In [165]:
#len(results[0])

530446

Saving the results

In [166]:
import pickle
# with open('levenshtein_results.pkl', 'wb') as f:
#     pickle.dump(results, f)

## Merging results to original data

In [ ]:
#Run this to load the saved results
file = open('levenshtein_results.pkl', 'rb')

results = pickle.load(file)

In [209]:
# The results will have 3 information: the matched laz item, score and match index
df = pd.DataFrame(zip(results[0], results[1], results[2]), columns=['match', 'score', 'match_index'])

In [210]:
# Data transformation
match = df['match'].apply(pd.Series).stack().reset_index()
score = df['score'].apply(pd.Series).stack().reset_index()
match_index = df['match_index'].apply(pd.Series).stack().reset_index()

In [219]:
# Merge the results columns together
res = match.merge(score, on=['level_0', 'level_1']).merge(match_index, on=['level_0', 'level_1'])

In [222]:
res.columns = ['item_index', 'match_no', 'match', 'score', 'match_index']

In [298]:
# Merge results with lazada data and shopee data
res_merged = res.merge(laz, left_on='match_index', right_index=True).sort_values(['item_index', 'match_no'])
res_merged = res_merged.drop(['item_description'], axis=1).rename(columns={'item_price':'match_price', 'final_weight':'match_weight'})

res_merged = res_merged.merge(spe, left_on='item_index', right_index=True).sort_values(['item_index', 'match_no'])

In [299]:
res_final = res_merged[['legacy_id', 'item_description', 'item_index', 'weight', 'price', 'quantity', 
                       'match', 'match_no', 'score', 'match_price', 'match_weight']]

In [300]:
# Calculate weight and price difference
res_final['weight_diff'] = res_final.match_weight - res_final.weight
res_final['price_diff'] = res_final.match_price - res_final.price

<ipython-input-300-834937784b45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_final['weight_diff'] = res_final.match_weight - res_final.weight


In [359]:
res_final.head()

,legacy_id,item_description,item_index,weight,price,quantity,match,match_no,score,match_price,match_weight,weight_diff,price_diff
0,303589,áo tập gym yoga nữ siêu xinh ami,0,0.15,190000,24,áo cổ v siêu xinh,0,56.000000,175000.0,0.316667,0.166667,-15000.0
1,303589,áo tập gym yoga nữ siêu xinh ami,0,0.15,190000,24,"lẻ 1 áo tập gym/ yoga siêu đẹp, áo bra thun tă...",1,55.913978,28000.0,0.230000,0.080000,-162000.0
2,303589,áo tập gym yoga nữ siêu xinh ami,0,0.15,190000,24,áo sơ mi trơn tay ngắn túi nắp,2,53.968254,61550.0,0.300000,0.150000,-128450.0
3,303589,áo tập gym yoga nữ siêu xinh ami,0,0.15,190000,24,áo sơ mi nữ tay ngắn lụa tằm qccc cực xinh xắn,3,53.164557,84400.0,0.300000,0.150000,-105600.0
4,303589,áo tập gym yoga nữ siêu xinh ami,0,0.15,190000,24,áo sơ mi nam kaki phối túi hộp siêu hot,4,52.777778,133000.0,0.300000,0.150000,-57000.0


## Aggregate by item and shipper level

By Item level: get the first (best) match, the average and the std dev for score, weight_diff and price_diff

In [301]:
res_gb_item = res_final.groupby(['legacy_id', 'item_description', 'item_index', 'weight', 'price', 'quantity'])\
                                [['score', 'weight_diff', 'price_diff']].agg(['first','mean','std'])

In [302]:
res_gb_item = res_gb_item.reset_index().sort_values('item_index')

Filter: here I want to keep only results where weight_diff and price_diff does not vary too much (<10kg and <50.000 VND) between the 5 matched items, to mitigate false positive cases

In [314]:
res_gb_item_filtered = res_gb_item[(res_gb_item[('weight_diff', 'std')]<=10)&(res_gb_item[('price_diff', 'mean')]<=50000)]#.nlargest(20, ('weight_diff', 'mean'))

By shipper level: get the average weight_diff and number of items for ranking

In [337]:
res_gb_shipper = res_gb_item_filtered.groupby('legacy_id').agg(['mean', 'count']).iloc[:,14:16].reset_index()

C:\Users\Home\anaconda3\lib\site-packages\pandas\core\generic.py:4152: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Filter: here I want to keep only shippers with more than 5 items. Below is the top 30 shippers according to the results

In [340]:
res_gb_shipper[res_gb_shipper[('weight_diff', 'first', 'count')]>5].nlargest(30, ('weight_diff', 'first', 'mean'))

legacy_id weight_diff      
                      first      
                       mean count
144      283677   15.400000     6
31854    518274   11.852778     6
10792    342800   11.032143     7
21041    420518    7.481500     6
10241    339095    7.433667    15
15213    374184    7.187917     6
9371     333459    7.006667     6
7111     319293    6.876563     8
9123     331768    6.498333     7
7139     319513    6.445431     6
18904    403826    6.294833     6
16657    385012    5.703333     6
11798    350855    5.546148     9
20719    418181    5.339259     9
28109    478869    5.132870     9
29131    488915    5.108661    14
4028     302160    5.064643     6
13962    365220    4.879933    15
19944    411903    4.843750     6
34384    536661    4.733333     6
24879    452064    4.667429     7
14684    370231    4.562561    19
9396     333617    4.546583     6
41137    558130    4.520833     6
2981     297778    4.480235    34
18138    397605    4.471953    16
6172     313719    4.463125    16
7932     324923    4.458865    12
27874    476799    4.453037     9
9403     333643    4.445000     6

Here you can paste the shipper legacy_id and see their selling items

In [349]:
res_gb_item[res_gb_item.legacy_id==333459]

legacy_id                                   item_description  \
                                                                      
197525    333459  vali kéo du lịch size 20,size 24 khung nhôm kh...   
197523    333459  vali kéo du lịch khung nhôm thời trang chống b...   
197522    333459  vali kéo du lịch khung nhôm khoá sập,vali khóa...   
197521    333459  vali du lịch khung nhôm, vali kéo size 20, siz...   
197526    333459  áo bọc vali silicon trong suốt, áo trùm vali c...   
197524    333459  vali kéo du lịch khung nhôm, vali khóa kéo chố...   

       item_index weight    price quantity      score                       \
                                                first       mean       std   
197525      28015   5.00  1250000       26  55.670103  51.416854  2.443566   
197523      77900   4.00  1249000        1  64.864865  56.090819  4.940671   
197522     149139   4.00   499000        3  58.333333  53.926788  3.681757   
197521     196131   6.00  1550000        1  56.994819  53.341439  2.111790   
197526     402538   0.01   119000       11  51.785714  49.919767  1.400075   
197524     482571   4.00   549000        1  57.777778  53.410814  2.526639   

       weight_diff                     price_diff                               
             first      mean       std      first          mean            std  
197525       10.00 -0.333400  6.076224  -851000.0 -9.820000e+05  204281.668292  
197523       11.00  3.078667  7.473176  -850000.0 -9.051880e+05  257327.955574  
197522        1.00  3.242000  7.288712        0.0 -1.546000e+05  256614.107173  
197521        9.00 -1.292765  6.286302 -1151000.0 -1.151454e+06  167972.041063  
197526        0.04  0.095000  0.059582   -74000.0 -5.857562e+04   37484.907276  
197524       11.00  3.140400  7.403273  -150000.0 -2.228000e+05  280491.220896

In [361]:
#res_gb_shipper.to_csv("shipper_list_levenshtein.csv", index=False)

### Examples

This section shows some examples for the algo

In [86]:
x = ['Máy Hút MụnMáy Hút Mụn Đầu Đenmàn hình led, máy hút mụn sạc pin 5 đầu hút SIÊU MẠNH']

Here is the 5 matched items using library with only heavy items

In [38]:
r = fuzzy(laz_items, x)

In [39]:
r

([['máy cắt sắt, máy cắt super - 8070 , máy cắt để bàn siêu bền, siêu tiện ích',
   'máy hút bụi gia đình có dây cực mạnh,máy hút bụi và hổi bụi gia đình supor',
   'máy hút mùi nhà bếp sunhouse shb6118b - máy hút khói được bảo hành 18 tháng tại nhà',
   'máy fr900- máy hàn miệng túi tự động có in hạn sử dụng - máy hàn máy in 2 trong 1',
   'máy chiết rót nén khí a02- máy chiết rót mỹ phẩm vòi inox siêu xịn']],
 [[48.40764331210191,
   48.40764331210191,
   48.19277108433735,
   47.5609756097561,
   46.97986577181208]])

Here is the 5 matched items using FULL library (very big, close to 200k items). We can see the accuracy is much better

In [59]:
r = fuzzy(laz_items, x)

In [60]:
r

([['máy hút mụn, máy hút mụn đầu đen, máy hút mụn cầm tay đa năng hàn quốc, máy hút mụn dùng pin sạc an toàn tiện dụng tặng kèm 4 đầu hút thông minh',
   'máy hút mụn cám, mụn đầu đen, mụn trứng cá - máy hút mụn cầm tay mini cao cấp 4 in 1 sử dụng cho mọi loại da',
   ' máy hút mụn đầu đen 5 lực hút - máy sạc pin hút mụn cải thiện làn da - đánh bay mụn',
   'máy hút mụn spa, hút mụn, máy rửa mặt - máy hút mụn trứng cá giá tốt',
   'máy hút mụn đầu đen, mụn cám, chăm sóc da mặt cao cấp với ba đầu hút']],
 [[61.67400881057269,
   59.68586387434555,
   57.83132530120482,
   56.95364238410596,
   55.629139072847686]])